In [19]:
import numpy as np
import math as mt
from itertools import combinations

In [20]:
def RotAngleLig(theta):
    matrix = np.array(
    [[-np.cos(theta), -np.sin(theta), 0, 0],
    [  np.sin(theta), -np.cos(theta), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
    ])
    return matrix

In [21]:
def RotAngleTor(beta):
    matrix = np.array(
    [[1, 0, 0, 0],
    [ 0, np.cos(beta), -np.sin(beta),  0],
    [ 0, np.sin(beta),  np.cos(beta), 0],
    [0, 0, 0, 1]
    ])
    return matrix

In [22]:
def Translacao(d):
    matrix = ([
    [1, 0, 0, d],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
    ])
    return matrix

In [23]:
def comb(theta, beta, d):
    return  RotAngleTor(beta)@RotAngleLig(theta)@Translacao(d)

In [24]:
print(comb(mt.pi, mt.pi/2, 0.2))

[[ 1.00000000e+00 -1.22464680e-16  0.00000000e+00  2.00000000e-01]
 [ 7.49879891e-33  6.12323400e-17 -1.00000000e+00  1.49975978e-33]
 [ 1.22464680e-16  1.00000000e+00  6.12323400e-17  2.44929360e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [25]:
def Positions(theta, beta, d):
    n = len(d)
   
    e4 = np.array([0, 0, 0, 1])
    
    positions = []

    Id = np.eye(4)
    
    for i in range(n):
        Bi = comb(theta[i], beta[i], d[i])
        Id = Id @ Bi
        xi = Id @ e4
        positions.append(xi[:3])  
    return np.array(positions)

In [26]:
def Distances(positions):
    n = len(positions)


    distance = {}

    for i, j in combinations(range(n), 2):
        rij = np.linalg.norm(positions[j] - positions[i])
        distance[(i+1, j+1)] = rij
    return distance

In [27]:
d = [0, 1.0, 1.0, 1.0]                # distâncias
theta = [np.pi, np.pi, np.pi/3, np.pi/4]  # ângulos de ligação
omega = [0, 0, 0, np.pi/6]            # ângulos de torção

In [58]:
e0 = np.array([[0],[0],[0],[1],[0]])
e_inf = np.array([[0],[0],[0],[0],[1]])


In [70]:
def C_matrix(theta, omega, d):
    """Matriz conforme 5x5 que combina rotação e translação."""
    A = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [ np.sin(theta)*np.cos(omega), -np.cos(theta)*np.cos(omega), -np.sin(omega)],
        [ np.sin(theta)*np.sin(omega), -np.cos(theta)*np.sin(omega),  np.cos(omega)]
    ])
    
    b = np.array([[-d*np.cos(theta)],
                  [ d*np.sin(theta)*np.cos(omega)],
                  [ d*np.sin(theta)*np.sin(omega)]])
   
    normb2 = float(np.sum(b**2))
    
   
    U = np.block([
        [A, b, np.zeros((3,1))],
        [np.zeros((1,3)), np.array([[1,0]])],
        [b.T, np.array([[normb2/2, 1]])]
    ])
    return U


In [82]:
def ConformalDistances(i, j):
    Bprod = np.eye(5)
    for k in range(i+1, j):
        Bprod = Bprod @ B[k]
    val = 2 * (e_inf.T @ Bprod @ e0)
    return val.item()




In [73]:
#coluna 3: d
#coluna 5: theta
#coluna 7: omega 

In [85]:
dist_conformal = {}
dist_euclidean = {}

for i, j in combinations(range(4), 2):
    rij2_conf = ConformalDistances(i, j)
    rij_conf = np.sqrt(rij2_conf)
    rij_euc = np.linalg.norm(x[j] - x[i])
    dist_conformal[(i+1, j+1)] = rij_conf
    dist_euclidean[(i+1, j+1)] = rij_euc

dist_conformal, dist_euclidean

({(1, 2): np.float64(0.0),
  (1, 3): np.float64(1.0),
  (1, 4): np.float64(1.0),
  (2, 3): np.float64(0.0),
  (2, 4): np.float64(1.0),
  (3, 4): np.float64(0.0)},
 {(1, 2): np.float64(0.0),
  (1, 3): np.float64(1.0),
  (1, 4): np.float64(0.9999999999999999),
  (2, 3): np.float64(1.0),
  (2, 4): np.float64(0.9999999999999999),
  (3, 4): np.float64(1.0)})

In [74]:
#Vamos ler o arquivo para podermos aplicar o metodo

def LerMat(arquivo):
    atoms = []
    variables = {}

    with open(arquivo, 'r') as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]

    split_index = None
    for i, l in enumerate(lines):
        if any(char.isdigit() for char in l.split()[0]):
            split_index = i
            break

    if split_index is None:
        split_index = len(lines)

    for line in lines[:split_index]:
        parts = line.split()
        n = len(parts)
        if n in [1, 3, 5, 7]:
            atoms.append(parts)
        else:
            print("Aviso: linha malformada:", parts)

    for line in lines[split_index:]:
        parts = line.split()
        if len(parts) == 2:
            name, value = parts
            try:
                variables[name] = float(value)
            except ValueError:
                pass

    return atoms, variables

# Uso
atoms = LerMat('/home/rnatieli/Downloads/we02.zmat')[0]
values = LerMat('/home/rnatieli/Downloads/we02.zmat')[1]


In [88]:
atoms, values

([['C'],
  ['H', '0', 'dist01'],
  ['H', '0', 'dist02', '1', 'ang01'],
  ['O', '1', 'dist03', '2', 'ang02', '0', 'ang03']],
 {'dist01': 1.09,
  'dist02': 1.09,
  'dist03': 1.43,
  'ang01': 109.5,
  'ang02': 104.5,
  'ang03': 120.0})